In [ ]:
NAME = "Akshay Pozath"
COLLABORATORS = ""

---


#  Twitter Data Analysis



In [ ]:

import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import json


pd.set_option('max_colwidth', 280)

%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re

In [ ]:
pip install tweepy

In [ ]:

import tweepy

## Accessing Twitter API  


### Reading Keys.json

In [ ]:
import json
key_file = 'keys.json'

with open(key_file) as f:
    keys = json.load(f)


: 

In [ ]:
import tweepy
from tweepy import TweepyException
import logging

try:
    auth = tweepy.OAuthHandler(keys["consumer_key"], keys["consumer_secret"])
    redirect_url = auth.get_authorization_url()
    auth.set_access_token(keys["access_token"], keys["access_token_secret"])
    api = tweepy.API(auth)
    print("Rutgers username is:", api.get_user(screen_name="RutgersU").name)
except TweepyException as e:
    logging.warning("There was a Tweepy error. Double check your API keys and try again.")
    logging.warning(e)



### Refactor and Extend Code


In [ ]:
def load_keys(path):
  

In [ ]:
def download_recent_tweets_by_user(user_account_name, keys):
   
    import tweepy
    
   

In [ ]:
def load_tweets(path):

In [ ]:
def get_tweets_with_cache(user_account_name, keys_path):

In [ ]:

rutgers_tweets = download_recent_tweets_by_user("RutgersU", key_file)
print("Number of tweets downloaded:", len(rutgers_tweets))

## Working with Twitter Data 


In [ ]:
from pathlib import Path
import json

ds_tweets_save_path = "/home/ap1877/shared/RutgersU_recent_tweets.json"   


if not Path(ds_tweets_save_path).is_file():
    
    example_tweets = [t._json for t in tweepy.Cursor(api.user_timeline, screen_name="RutgersU", 
                                             tweet_mode='extended').items()]
    
    
    with open(ds_tweets_save_path, "w") as f:        
        json.dump(example_tweets, f)


with open(ds_tweets_save_path, "r") as f:
    example_tweets = json.load(f)

In [ ]:
def load_tweets(path):
    """Loads tweets that have previously been saved.
    
    Calling load_tweets(path) after save_tweets(tweets, path)
    will produce the same list of tweets.
    
    Args:
        path (str): The place where the tweets will be saved.

    Returns:
        list: A list of Dictionary objects, each representing one tweet."""
    
    with open(path, "rb") as f:
        import json
        return json.load(f)

In [ ]:
dest_path = "/home/ap1877/shared/TrumpTweets_1.json"
trump_tweets = load_tweets(dest_path)

In [ ]:

for i, tweet in enumerate(trump_tweets[:10], 1):
    print(f'Tweet {i}:')
    print(f'Text: {tweet.get("full_text", "N/A")}')
    print('---')


### Oldest Tweet



In [ ]:

trump_tweets = pd.DataFrame(trump_tweets)


def oldest_tweet(df):
    month = df['created_at'].str[4:7]
    month = month.rename('month')
    year = df['created_at'].str[-4:]
    year = year.rename('year')

    oldest_month = pd.concat([month,year], axis=1)
    minyear = oldest_month['year'].min()
    oldest_month = oldest_month[oldest_month['year']=='2017']
    m = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 
     'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

    oldest_month['month'] = oldest_month['month'].map(m)
    minmonth = oldest_month['month'].min()

    return (minmonth)


oldest_tweet_month = oldest_tweet(trump_tweets)
print(oldest_tweet_month)

## Twitter Source Analysis 



### Merge the dataframes



In [ ]:

path2 =  "/home/ap1877/shared/TrumpTweets_2.json"
trump_tweets2 = load_tweets(path2)
trump_tweets2 = pd.DataFrame(trump_tweets2)

path3 =  "/home/ap1877/shared/TrumpTweets_3.json"
trump_tweets3 = load_tweets(path3)
trump_tweets3 = pd.DataFrame(trump_tweets3)

all_tweets = pd.concat([trump_tweets, trump_tweets2, trump_tweets3]).drop_duplicates(subset='id', keep='first').reset_index(drop=True)
### END ANSWER

all_tweets.head()

### Constructing DataFrame



In [ ]:

all_tweets = all_tweets.astype({'full_text':'str'})
text = []
date = []


for index, row in all_tweets.iterrows():
    if row['full_text']=='nan':
        text.append(row['full_text'])
    else:
        text.append(row['full_text'])
    date.append(pd.to_datetime(row['created_at']))
df_trump = all_tweets[['id', 'source','retweet_count','favorite_count']]
df_trump.insert(1, "time", date, True)
df_trump.insert(3, "text", text, True)
df_trump = df_trump.astype({'id':'str'})
df_trump = df_trump.sort_values(by=['id'])
df_trump = df_trump.set_index('id')
df_trump = df_trump.rename(columns={"created_at": "time"})
df_trump = df_trump[~df_trump.index.duplicated(keep='first')]
df_trump


In [ ]:
df_trump['source'].unique()

In [ ]:
import re

   
df_trump['source'] = df_trump['source'].str.replace(r'<[^>]*>','')
df_trump


####  Most Common Device

In [ ]:

   
device_list = df_trump['source']
device_list = device_list.groupby(device_list)
device_list = device_list.count()
device_list
keyList = device_list.keys().to_list()
valList = device_list.to_list()

df_v2 = pd.DataFrame(valList, columns=['counts'], index=keyList)
df_v2 = df_v2.sort_values(by='counts', ascending=True)
ax = df_v2.plot.barh(legend=False)
ax.set_xlabel('Number of Tweets')
print(df_v2)


### Time Analysis

In [ ]:
df_trump['time'][0:3]

In [ ]:
df_trump['est_time'] = (
    df_trump['time'] # Set initial timezone to UTC
                 .dt.tz_convert("EST") # Convert to Eastern Time
)
df_trump.head()

In [ ]:
df_trump['hour'] =  df_trump['time'].dt.hour + df_trump['time'].dt.minute/60 + df_trump['time'].dt.second/3600
df_trump['roundhour']=round(df_trump['hour'])

In [ ]:

thehour = df_trump['roundhour'].groupby(df_trump['roundhour'])
thehour = thehour.count()
thehour[24] = thehour[24]+thehour[0]
thehour = thehour.drop(0)
thehour

fig, ax = plt.subplots(1,1, figsize=(15,10))
sns.barplot(x=thehour.index, y=thehour.values, orient='v')
ax.tick_params(axis='x', rotation=90)
ax.set_title('Times that Trump uses Twitter')
ax.set_ylabel('Count of Tweets')
ax.set_xlabel('Hour of day')
fig.show()


In [ ]:

   
top2devices = devices.nlargest(2).keys()
dist1 = df_trump[df_trump['source'] == top2devices[0]]
dist2 = df_trump[df_trump['source'] == top2devices[1]]
fig, ax = plt.subplots(1,1)

sns.distplot(dist1['hour'], hist=False, kde=True)
sns.distplot(dist2['hour'], hist=False, kde=True)

fig.legend(labels=[top2devices[0][12:],top2devices[1][12:]])
plt.xlabel("Hour")
plt.ylabel("Fraction")
plt.show()


### iPhone or Android




In [ ]:

   
dist1 = dist1[dist1['time'].dt.year == 2016]
dist2 = dist2[dist2['time'].dt.year == 2016]

fig, ax = plt.subplots(1,1)

sns.distplot(dist1['hour'], hist=False, kde=True)
sns.distplot(dist2['hour'], hist=False, kde=True)

fig.legend(labels=[top2devices[0][12:],top2devices[1][12:]])
plt.xlabel("Hour")
plt.ylabel("Fraction")
plt.show()


### Device Analysis


In [ ]:
import datetime
def year_fraction(date):
    start = datetime.date(date.year, 1, 1).toordinal()
    year_length = datetime.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length


df_trump['year'] = df_trump['time'].apply(year_fraction) #should be df_trump

In [ ]:


dist1 = df_trump[df_trump['source'] == top2devices[0]]
dist2 = df_trump[df_trump['source'] == top2devices[1]]
fig, ax = plt.subplots(1,1)

sns.distplot(dist1['year'], kde=True)
sns.distplot(dist2['year'], kde=True)
fig.legend(labels=[top2devices[0][12:],top2devices[1][12:]])


plt.xlabel("year")
plt.show()



##  Sentiment Analysis 



In [ ]:
print(''.join(open("/home/ap1877/shared/vader_lexicon.txt").readlines()[:10]))

: 

### Polarities


In [ ]:


inde = []
value = []
for line in open("/home/ap1877/shared/vader_lexicon.txt").readlines():
    temp = re.split(r'\t+', line)
    inde.append(temp[0])
    value.append(float(temp[1]))
df_sent = pd.DataFrame(value, index=inde, columns=['polarity'])
df_sent



### Sentiment

Use lexicon

-For each tweet, find the sentiment of each word.
-Sum of sentiments



In [ ]:

   
df_trump['text'] = df_trump['text'].str.lower()
df_trump


### punctuations




In [ ]:
# Save your regex in punct_re
punct_re = r'[^\w\s\\n]'

regex = re.compile(punct_re)
df_trump['no_punc'] = df_trump['text'].replace(regex,' ')
df_trump.index = pd.to_numeric(df_trump.index)
df_trump



In [ ]:
assert isinstance(punct_re, str)
assert re.search(punct_re, 'this') is None
assert re.search(punct_re, 'this is ok') is None
assert re.search(punct_re, 'this is\nok') is None
assert re.search(punct_re, 'this is not ok.') is not None
assert re.search(punct_re, 'this#is#ok') is not None
assert re.search(punct_re, 'this^is ok') is not None
assert df_trump['no_punc'].loc[800329364986626048] == 'i watched parts of nbcsnl saturday night live last night it is a totally one sided biased show nothing funny at all equal time for us'
assert df_trump['text'].loc[884740553040175104] == 'working hard to get the olympics for the united states (l.a.). stay tuned!'


### Tidy Format

In [ ]:

   
tidy_format = df_trump['no_punc'].str.split(expand=True)
tidy_format = tidy_format.stack()
tidy_format = tidy_format.reset_index()
tidy_format.columns = ['ID', 'num', 'word']


print(tidy_format)



In [ ]:
assert tidy_format.loc[894661651760377856].shape == (27, 2)
assert ' '.join(list(tidy_format.loc[894661651760377856]['word'])) == 'i think senator blumenthal should take a nice long vacation in vietnam where he lied about his service so he can at least say he was there'

### Polrity 



In [ ]:


senti = tidy_format.merge(df_sent, how='left', left_on='word', right_index=True)

senti = senti.groupby('ID').sum()
df_trump['polarity'] = senti['polarity']



df_trump.head()

: 

In [ ]:
assert np.allclose(df_trump.loc[744701872456536064, 'polarity'], 8.4)
assert np.allclose(df_trump.loc[745304731346702336, 'polarity'], 2.5)
assert np.allclose(df_trump.loc[744519497764184064, 'polarity'], 1.7)
assert np.allclose(df_trump.loc[894661651760377856, 'polarity'], 0.2)
assert np.allclose(df_trump.loc[894620077634592769, 'polarity'], 5.4)



### Positive and Negative Tweets


In [ ]:
print('Most negative tweets:')


neg = df_trump[['polarity', 'text']]
neg = neg.sort_values('polarity', ascending=True)
neg = neg.head(5)
neg = neg['text']
neg



: 

In [ ]:
print('Most positive tweets:')


   
pos = df_trump[['polarity', 'text']]
pos = pos.sort_values('polarity', ascending=False)
pos = pos.head(5)
pos = pos['text']
pos


### Comparisons

In [ ]:

fox = df_trump[df_trump['text'].str.contains('fox')]
nyt = df_trump[df_trump['text'].str.contains('nyt')]
fig, ax = plt.subplots(1,1)

sns.distplot(nyt['polarity'], kde=True)
sns.distplot(fox['polarity'], kde=True)

fig.legend(labels=['nyt','fox'])
plt.show()


## Principal Component Analysis (PCA)

In [ ]:
pip install nlkt

In [ ]:


import nltk
import nltk.corpus
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

tmp = tidy_format.drop('num',axis=1)
tmp = tmp.set_index(tmp.index)
tmp


stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend(['rt','t','co','https','realdonaldtrump','amp',"u",'hillary','trump2016','trump','clinton','http','ha','wa'])
tmp = tmp[~tmp['word'].isin(stopwords)]


from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
def lem(x):
    return lemma.lemmatize(x)
tmp['word'] = tmp.word.apply(lem)

tmp = tmp[~(tmp['word'].str.isnumeric())]


tmp = tmp[(tmp['word'].str.len() > 2)]

tmp = tmp.reset_index()
fifty = tmp['word'].value_counts(ascending=False).nlargest(50).to_frame()
tmp2 = tmp[tmp['word'].isin(fifty.reset_index()['index'])]
ids = tmp2['ID'].unique()
ids = ids[:5000]
matrix = np.zeros((5000,50))
fifty = fifty.reset_index()

for a in range(5000):
    for b in range(50):
        if (fifty['index'][b]) in df_trump['no_punc'].loc[ids[a]]:
            matrix[a][b] += 1

matrix[:10]


In [ ]:
pip install scikit-learn

In [ ]:

   
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(matrix)
print(pca.explained_variance_ratio_)




### Examine the PCA


In [ ]:


heatmap_df = pd.DataFrame(pca.components_)   
heatmap_df['word'] = fifty['index']
heatmap_df = heatmap_df.set_index('word')
xlist = []
for x in range(50):
    xlist.append("PC" + str(x+1))

fig, ax = plt.subplots(1,1, figsize=(15,10))
sns.heatmap(heatmap_df, xticklabels=xlist, yticklabels=True, cmap="YlGn")


### Compare PCA

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
   
graph = sns.JointGrid(data=heatmap_df, x = 1, y = 2)
graph.plot(sns.scatterplot, sns.histplot)

ax_joint = graph.ax_joint
ax_marg_x = graph.ax_marg_x
ax_marg_y = graph.ax_marg_y

ax_joint.set_xticks([0.2,0, 0.2, 0.4, 0.6]) 
ax_joint.set_xlim(-0.2, 0.6)  


ax_joint.set_yticks([-0.2, 0, 0.2, 0.4, 0.6,0.8])
ax_joint.set_ylim(-0.3, 0.8)

graph.set_axis_labels('PC1', 'PC2')


plt.show()
